In [10]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [168]:
import xlrd
import plotly.express as px
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import plan_utils as utils
import plan_var as V
from plan_loader import *
from reformatter import *

In [197]:
%autoreload
plan_loader = PlanLoader('план.xlsx')
plan_loader.delete_rows_ready()

In [246]:
%autoreload
ref = PlanReformatter(plan_loader.df)
ref.r_df

,Заливка_НМ_Нач,Заливка_НМ_Кон,Заливка_ВМ_Нач,Заливка_ВМ_Кон,Навеска_НМ_Нач,Навеска_НМ_Кон,Навеска_ВМ_Нач,Навеска_ВМ_Кон,ЭМУ_Нач,ЭМУ_Кон,...,УсткаВВ_Нач,УсткаВВ_Кон,УсткаRM_Нач,УсткаRM_Кон,Наладка_Нач,Наладка_Кон,Комплект_Нач,Комплект_Кон,ОТГРУЗКА,ОТГРУЗКА_1
1250,готов,готов,готов,готов,2019-11-20 00:00:00,2019-11-20 00:00:00,2019-11-20 00:00:00,2019-11-25 00:00:00,готов,готов,...,2019-11-19 00:00:00,2019-11-19 00:00:00,2019-11-20 00:00:00,2019-11-20 00:00:00,2019-12-05 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-06 00:00:00,2019-12-06,нет
1253,готов,готов,готов,готов,2019-11-26 00:00:00,2019-11-26 00:00:00,2019-11-26 00:00:00,2019-11-27 00:00:00,2019-11-25 00:00:00,2019-11-25 00:00:00,...,2019-12-25 00:00:00,2019-12-25 00:00:00,2019-12-27 00:00:00,2019-12-27 00:00:00,2019-12-05 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-06 00:00:00,2019-12-10,нет
1259,нет,нет,нет,нет,нет,нет,нет,нет,нет,нет,...,нет,нет,нет,нет,2019-12-09 00:00:00,2019-12-09 00:00:00,2019-12-09 00:00:00,2019-12-09 00:00:00,2019-12-10,нет
1255,готов,готов,готов,готов,2019-11-28 00:00:00,2019-11-28 00:00:00,2019-11-28 00:00:00,2019-11-30 00:00:00,2019-11-26 00:00:00,2019-11-26 00:00:00,...,2019-11-29 00:00:00,2019-11-29 00:00:00,2019-12-03 00:00:00,2019-12-03 00:00:00,2019-12-09 00:00:00,2019-12-09 00:00:00,2019-12-10 00:00:00,2019-12-10 00:00:00,2019-12-11,нет
1256,готов,готов,готов,готов,2019-12-02 00:00:00,2019-12-02 00:00:00,2019-12-02 00:00:00,2019-12-04 00:00:00,2019-11-29 00:00:00,2019-11-29 00:00:00,...,2019-12-03 00:00:00,2019-12-03 00:00:00,2019-12-03 00:00:00,2019-12-03 00:00:00,2019-12-10 00:00:00,2019-12-10 00:00:00,2019-12-11 00:00:00,2019-12-11 00:00:00,2019-12-12,нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,2020-09-28 00:00:00,2020-09-30 00:00:00,2020-10-06 00:00:00,2020-10-12 00:00:00,2020-10-28 00:00:00,2020-10-28 00:00:00,2020-10-28 00:00:00,2020-10-30 00:00:00,2020-10-28 00:00:00,2020-10-28 00:00:00,...,2020-10-30 00:00:00,2020-10-30 00:00:00,2020-10-30 00:00:00,2020-10-30 00:00:00,2020-11-06 00:00:00,2020-11-06 00:00:00,2020-11-09 00:00:00,2020-11-09 00:00:00,2020-11-10,нет
1338,2020-09-30 00:00:00,2020-10-02 00:00:00,2020-10-13 00:00:00,2020-10-16 00:00:00,нет,нет,нет,нет,нет,нет,...,нет,нет,нет,нет,нет,нет,нет,нет,2020-11-18,нет
1335,2020-10-05 00:00:00,2020-10-13 00:00:00,2020-10-16 00:00:00,2020-10-26 00:00:00,2020-11-02 00:00:00,2020-11-02 00:00:00,2020-11-02 00:00:00,2020-11-06 00:00:00,2020-11-03 00:00:00,2020-11-03 00:00:00,...,2020-10-05 00:00:00,2020-11-02 00:00:00,нет,нет,2020-11-17 00:00:00,2020-11-19 00:00:00,2020-11-19 00:00:00,2020-11-19 00:00:00,2020-11-20,нет
1339,2020-10-13 00:00:00,2020-10-16 00:00:00,2020-10-26 00:00:00,2020-10-29 00:00:00,нет,нет,нет,нет,нет,нет,...,нет,нет,нет,нет,нет,нет,нет,нет,2020-11-26,нет


In [224]:
i=0
l = []
while i < len(ColumnNames)-2:
    l.append(ColumnNames[i][:-4])
    i+=2
l

['Заливка_НМ',
 'Заливка_ВМ',
 'Навеска_НМ',
 'Навеска_ВМ',
 'ЭМУ',
 'СборкаЩитов',
 'Монтаж_СН',
 'ШинаУВР',
 'Шина_ТР',
 'СборкаНН',
 'ВторНН',
 'СборкаВВ',
 'ВторВВ',
 'УсткаНН',
 'УсткаВВ',
 'УсткаRM',
 'Наладка',
 'Комплект']

In [272]:
%autoreload
data = ref.get_df_for_gantt(22)
data

,Task,Start,Finish,zakaz
0,Монтаж_СН,2020-09-07 08:20:00,2020-09-08 16:20:00,1324
1,ВторНН,2020-09-01 08:20:00,2020-09-08 16:20:00,1324
2,УсткаНН,2020-09-02 08:20:00,2020-09-02 16:20:00,1324
3,УсткаВВ,2020-09-02 08:20:00,2020-09-02 16:20:00,1324
4,УсткаRM,2020-09-02 08:20:00,2020-09-02 16:20:00,1324
...,...,...,...,...
229,Комплект,2020-11-19 08:20:00,2020-11-19 16:20:00,1335
230,Заливка_НМ,2020-10-13 08:20:00,2020-10-16 16:20:00,1339
231,Заливка_ВМ,2020-10-26 08:20:00,2020-10-29 16:20:00,1339
232,Заливка_НМ,2020-10-16 08:20:00,2020-10-21 16:20:00,1340


In [271]:
fig = px.timeline(data, x_start="Start", x_end="Finish", y="Task", color='zakaz', category_orders={'Task':l})
#fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.update_xaxes(
    dtick='D1',
    )
fig.show()

In [263]:
data[data['Task'] == 'УсткаRM']

,Task,Start,Finish,zakaz
4,УсткаRM,2020-09-02 08:20:00,2020-09-02 16:20:00,1324
11,УсткаRM,2020-09-02 08:20:00,2020-09-02 16:20:00,1325
15,УсткаRM,2020-09-10 08:20:00,2020-09-10 16:20:00,1322
27,УсткаRM,2020-09-11 08:20:00,2020-09-11 16:20:00,1320
41,УсткаRM,2020-09-18 08:20:00,2020-09-18 16:20:00,1321
70,УсткаRM,2020-09-29 08:20:00,2020-09-29 16:20:00,1329
85,УсткаRM,2020-10-02 08:20:00,2020-10-02 16:20:00,1327
101,УсткаRM,2020-10-09 08:20:00,2020-10-09 16:20:00,1331
136,УсткаRM,2020-10-17 08:20:00,2020-10-17 16:20:00,1332
159,УсткаRM,2020-10-19 08:20:00,2020-10-19 16:20:00,1333


In [266]:
data[data['zakaz'] == 1342]

,Task,Start,Finish,zakaz
104,Заливка_НМ,2020-09-16 08:20:00,2020-09-21 16:20:00,1342
105,Навеска_НМ,2020-09-18 08:20:00,2020-09-21 16:20:00,1342


In [256]:
data.iloc[104]['zakaz'] = 1342
data[data['zakaz'] == '1342 НМ']

AttributeError: 'int' object has no attribute 'iloc'